# Deep Learning Fundamentals 10 - Customizing the Neural Network

Welcome to another Notebook on Deep Learning Fundamentals. In this notebook, we will explore how to customize our model using customized loss functions, metrics, optimizers, etc...

In [1]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# Customizing our Model with Tensorflow

In this part, we will explore how we can use some of the Tensorflow data structures and operations.

## Custom Loss Functions

Let's say that we have a noisy data which also includes lots of outliers. Jut as we can use robust regression with Huber loss we can also use it in neural networks.

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

Let's implement the huber loss from scratch

In [3]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [4]:
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",input_shape=input_shape),
    keras.layers.Dense(12, activation="selu", kernel_initializer="lecun_normal",input_shape=input_shape),
    keras.layers.Dense(1),
])

In [5]:
model

Let's pass the loss function we implemented above while compiling the model.

In [6]:
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

In [7]:
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 0.4126 - mae: 0.7428 - val_loss: 0.1989 - val_mae: 0.4881
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1967 - mae: 0.4846 - val_loss: 0.1970 - val_mae: 0.4793
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.1852 - mae: 0.4660 - val_loss: 0.1785 - val_mae: 0.4537
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.1767 - mae: 0.4533 - val_loss: 0.1615 - val_mae: 0.4332
Epoch 5/10
363/363 [==============================] - 1s 4ms/step - loss: 0.1706 - mae: 0.4426 - val_loss: 0.1625 - val_mae: 0.4346
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.1679 - mae: 0.4379 - val_loss: 0.1841 - val_mae: 0.4495
Epoch 7/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1655 - mae: 0.4337 - val_loss: 0.1685 - val_mae: 0.4296
Epoch 8/10
363/363 [==============================] - 1s 2ms/step - loss: 0.

We can also use Huber loss without implementing it.

In [8]:
model.compile(loss=keras.losses.Huber(), optimizer="nadam", metrics=["mae"])

In [9]:
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1576 - mae: 0.4192 - val_loss: 0.1536 - val_mae: 0.4107
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1565 - mae: 0.4170 - val_loss: 0.1645 - val_mae: 0.4221
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1554 - mae: 0.4152 - val_loss: 0.1510 - val_mae: 0.4109
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1540 - mae: 0.4124 - val_loss: 0.1501 - val_mae: 0.4036
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1527 - mae: 0.4101 - val_loss: 0.1545 - val_mae: 0.4096
Epoch 6/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1529 - mae: 0.4096 - val_loss: 0.1538 - val_mae: 0.4092
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1516 - mae: 0.4082 - val_loss: 0.1509 - val_mae: 0.4077
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.

It's important to note that one should implement functions in vectorized format because it provides better performance. Moreover, Tensorflow's graph features can only be used when the implementation is done with Tensorflow operations.

Let's save the model and reload it.

In [10]:
model.save("my_model_with_a_custom_loss.h5")

In [11]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [12]:
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1483 - mean_absolute_error: 0.4010 - val_loss: 0.1471 - val_mean_absolute_error: 0.3980
Epoch 2/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1476 - mean_absolute_error: 0.4010 - val_loss: 0.1437 - val_mean_absolute_error: 0.3934
Epoch 3/10
363/363 [==============================] - 1s 1ms/step - loss: 0.1470 - mean_absolute_error: 0.3994 - val_loss: 0.1427 - val_mean_absolute_error: 0.3907
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1462 - mean_absolute_error: 0.3976 - val_loss: 0.1540 - val_mean_absolute_error: 0.4051
Epoch 5/10
363/363 [==============================] - 1s 4ms/step - loss: 0.1460 - mean_absolute_error: 0.3972 - val_loss: 0.1409 - val_mean_absolute_error: 0.3873
Epoch 6/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1450 - mean_absolute_error: 0.3952 - val_loss: 0.1521 - val_mean_absolute_error: 0.4015
Epoch 7/10
363/3

When we implement the loss function from scratch we can tune it, for instance, we can tune the threshold of huber loss for using alternatives of the loss function.

In [13]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [14]:
model.compile(loss=create_huber(1.4), optimizer="nadam", metrics=["mae"])

Now we can tune the threshold as well.

In [15]:
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
363/363 [==============================] - 1s 4ms/step - loss: 0.1519 - mae: 0.3894 - val_loss: 0.1488 - val_mae: 0.3952
Epoch 2/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1510 - mae: 0.3890 - val_loss: 0.1694 - val_mae: 0.3936
Epoch 3/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1512 - mae: 0.3884 - val_loss: 0.1487 - val_mae: 0.3853
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1498 - mae: 0.3860 - val_loss: 0.1587 - val_mae: 0.3836
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1492 - mae: 0.3848 - val_loss: 0.1552 - val_mae: 0.3847
Epoch 6/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1489 - mae: 0.3842 - val_loss: 0.1555 - val_mae: 0.3867
Epoch 7/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1475 - mae: 0.3828 - val_loss: 0.1520 - val_mae: 0.3792
Epoch 8/10
363/363 [==============================] - 1s 2ms/step - loss: 0.

We can save this model just as we did the previous one, however, the threshold will not be saved. Therefore, we need to keep it.

In [16]:
model.save("my_model_with_a_custom_loss_threshold_2.h5")

In [17]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [18]:
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1510 - mean_absolute_error: 0.3806 - val_loss: 0.1875 - val_mean_absolute_error: 0.4003
Epoch 2/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1518 - mean_absolute_error: 0.3804 - val_loss: 0.1487 - val_mean_absolute_error: 0.3732
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1496 - mean_absolute_error: 0.3793 - val_loss: 0.1943 - val_mean_absolute_error: 0.4008
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.1500 - mean_absolute_error: 0.3792 - val_loss: 0.1535 - val_mean_absolute_error: 0.3877
Epoch 5/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1492 - mean_absolute_error: 0.3778 - val_loss: 0.1725 - val_mean_absolute_error: 0.3807
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.1488 - mean_absolute_error: 0.3767 - val_loss: 0.1637 - val_mean_absolute_error: 0.3856
Epoch 7/10
363/3

We can solve the problem of unsaved threshold by creating a subclass in `keras.losses.Loss` class.

In [19]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

We edited the get_config() method so that when we save the model, Keras will call the loss's get_config() and the threshold will also be saved in the HDF5 file.

In [20]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [21]:
model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

In [22]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.8451 - mae: 0.9947 - val_loss: 0.3161 - val_mae: 0.5481
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2469 - mae: 0.5145 - val_loss: 0.2219 - val_mae: 0.4866


Let's save the model to see whether it will save the threshold or not.

In [23]:
model.save("my_model_with_a_custom_loss_class.h5")

In [24]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

In [25]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2301 - mean_absolute_error: 0.4988 - val_loss: 0.2216 - val_mean_absolute_error: 0.4799
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2249 - mean_absolute_error: 0.4918 - val_loss: 0.2260 - val_mean_absolute_error: 0.4804


In [26]:
model.loss.threshold

2.0

It worked!! When we load the model `from_config()` method was called and passed the config to the constructor.

## Customizing our Neural Network Further

Just as we defined custom cost functions we can also define custom Activation Functions, Initializers, Regularizers and Constraints.

In [27]:
keras.backend.clear_session()

In [28]:
def my_softplus(z): # this is equivalent to keras.activations.softplus() 
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32): # equivalent to keras.initializers.glorot_normal()
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights): # equivalent to keras.regularizers.l1(0.01)
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # equivalent to keras.constraints.nonneg() / ensures that weights are positive
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [29]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=my_l1_regularizer,
                       kernel_constraint=my_positive_weights,
                       kernel_initializer=my_glorot_initializer),
])

* The layer’s weights will be initialized using the value returned by the initializer. At each training step the weights will be passed to the regularization function to compute the regularization loss, which will be added to the main loss to get the final loss used for training. Finally, the constraint function will be called after each training step, and the layer’s weights will be replaced by the constrained weights. - [Géron, A. (2019)](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

In [30]:
model.compile(loss="mse", optimizer="nadam", metrics=["mae"])

In [31]:
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 1.6761 - mae: 0.9069 - val_loss: 0.9411 - val_mae: 0.5377
Epoch 2/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5785 - mae: 0.5191 - val_loss: 1.0967 - val_mae: 0.4974
Epoch 3/10
363/363 [==============================] - 1s 1ms/step - loss: 0.5182 - mae: 0.4948 - val_loss: 0.8587 - val_mae: 0.4869
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5017 - mae: 0.4882 - val_loss: 0.8335 - val_mae: 0.4751
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4940 - mae: 0.4840 - val_loss: 0.7462 - val_mae: 0.4802
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4883 - mae: 0.4833 - val_loss: 1.0155 - val_mae: 0.4809
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4860 - mae: 0.4823 - val_loss: 0.6261 - val_mae: 0.4694
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.

Let's also save the model

In [32]:
model.save("my_model_with_many_custom_parts.h5")

Let's load the model.

In [33]:
model = keras.models.load_model(
    "my_model_with_many_custom_parts.h5",
    custom_objects={
       "my_l1_regularizer": my_l1_regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

In a case in which we would like to save a hyperparameter we again need to use subclass API as we needed for the threshold of the custom loss function. The code below defines an L1 regularizer that saves its factor.

In [34]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

In [35]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=MyL1Regularizer(0.01),
                       kernel_constraint=my_positive_weights,
                       kernel_initializer=my_glorot_initializer),
])

In [36]:
model.compile(loss="mse", optimizer="nadam", metrics=["mae"])

In [37]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 4ms/step - loss: 2.0613 - mae: 0.9971 - val_loss: inf - val_mae: inf
Epoch 2/2
363/363 [==============================] - 2s 4ms/step - loss: 0.7937 - mae: 0.5485 - val_loss: inf - val_mae: inf


In [38]:
model.save("my_model_with_many_custom_parts.h5")

In [39]:
model = keras.models.load_model(
    "my_model_with_many_custom_parts.h5",
    custom_objects={
       "MyL1Regularizer": MyL1Regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

## Custom Metrics

Defining a custom metric is very similar to defining a custom loss function. In fact, we can even use the custom loss function as the metric. Nevertheless, it should be noted that losses and metrics are not the same thing.

* Losses and metrics are conceptually not the same thing: losses (e.g., cross entropy) are used by Gradient Descent to train a model, so they must be differentiable (at least where they are evaluated), and their gradients should not be 0 everywhere. Plus, it’s OK if they are not easily interpretable by humans. In contrast, metrics (e.g., accuracy) are used to evaluate a model: they must be more easily interpretable, and they can be non-differentiable or have 0 gradients everywhere - [Géron, A. (2019)](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

In [40]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [41]:
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=[create_huber(2.0)])

In [42]:
sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled, y_train, epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 0.4460 - huber_fn: 0.8904
Epoch 2/2
363/363 [==============================] - 0s 914us/step - loss: 0.1241 - huber_fn: 0.2488


Even though we used the same function for loss and metric, we got different results. There mainly two reasons for this

* the loss since the start of the epoch is the mean of all batch losses seen so far. Each batch loss is the sum of the weighted instance losses divided by the _batch size_ (not the sum of weights, so the batch loss is _not_ the weighted mean of the losses).
* the metric since the start of the epoch is equal to the sum of weighted instance losses divided by sum of all weights seen so far. In other words, it is the weighted mean of all the instance losses. Not the same thing.
* If you do the math, you will find that loss = metric * mean of sample weights (plus some floating point precision error).

Let's see if loss=metric*mean of sample weights actually holds.

In [43]:
history.history["loss"][0], history.history["huber_fn"][0] * sample_weight.mean()

(0.44603508710861206, 0.4431841325972598)

There is a small difference between the values, that's because, we also need to add some floating point precision error to loss . However, this is not that important for this example. 

Let's create a streaming Huber Loss ( A streaming metric is a metric that keeps track of the total Huber loss and the number of instances seen).

In [44]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [45]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [46]:
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=[HuberMetric(2.0)])

In [47]:
model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 0.7575 - huber_metric: 0.7575
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2426 - huber_metric: 0.2426


In [48]:
model.save("my_model_with_a_custom_metric.h5")

In [49]:
model = keras.models.load_model("my_model_with_a_custom_metric.h5",
                                custom_objects={"huber_fn": create_huber(2.0),
                                                "HuberMetric": HuberMetric})

## Custom Layers

In some cases we need to define our custom layers, for instance, if we have some repeated layers in our neural network we can create a custom layer Z that contains repeated layers.

Let's create a simple custom layer with no weights (such as keras.layers.Flatten or keras.layers.ReLU).

In [50]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [51]:
exponential_layer([-1., 0., 1.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.36787945, 1.        , 2.7182817 ], dtype=float32)>

The layer takes exponential of its input. We may need a layer like that for some regression problems in which the target value has very different scales such as 0.0001, 10, 1.000.

In [52]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=input_shape),
    keras.layers.Dense(1),
    exponential_layer
])
model.compile(loss="mse", optimizer="sgd")
model.fit(X_train_scaled, y_train, epochs=5,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/5
363/363 [==============================] - 1s 3ms/step - loss: 0.7413 - val_loss: 0.4277
Epoch 2/5
363/363 [==============================] - 1s 2ms/step - loss: 0.6277 - val_loss: 0.5022
Epoch 3/5
363/363 [==============================] - 1s 2ms/step - loss: 0.4573 - val_loss: 0.3600
Epoch 4/5
363/363 [==============================] - 1s 1ms/step - loss: 0.3999 - val_loss: 0.3530
Epoch 5/5
162/162 [==============================] - 0s 518us/step - loss: 0.3730


0.37297534942626953

Building a stateful layer (layer with weights) is somewhat harder that building a lambda layer. In that case, we need to subclass the `keras.layers.Layer` class. For instance, let's create a simple version of Dense layer.

In [53]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [54]:
model = keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=input_shape),
    MyDense(1)
])

In [55]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 1.5354 - val_loss: 4.8629
Epoch 2/2
162/162 [==============================] - ETA: 0s - loss: 0.512 - 0s 3ms/step - loss: 0.5099


0.5098922252655029

Creating a layer with multiple input/output is also quite similar. In this case, we need to use tuples for input and output.

In [56]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        print("X1.shape: ", X1.shape ," X2.shape: ", X2.shape) # Debugging of custom layer
        return X1 + X2, X1 * X2

    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]

Let's split the data and create a neural network which uses `MultiLayer()` model.

In [57]:
def split_data(data):
    columns_count = data.shape[-1]
    half = columns_count // 2
    return data[:, :half], data[:, half:]

X_train_scaled_A, X_train_scaled_B = split_data(X_train_scaled)
X_valid_scaled_A, X_valid_scaled_B = split_data(X_valid_scaled)
X_test_scaled_A, X_test_scaled_B = split_data(X_test_scaled)

# Printing the splitted data shapes
X_train_scaled_A.shape, X_train_scaled_B.shape

((11610, 4), (11610, 4))

In [58]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

input_A = keras.layers.Input(shape=X_train_scaled_A.shape[-1])
input_B = keras.layers.Input(shape=X_train_scaled_B.shape[-1])
hidden_A, hidden_B = MyMultiLayer()((input_A, input_B))
hidden_A = keras.layers.Dense(30, activation='selu')(hidden_A)
hidden_B = keras.layers.Dense(30, activation='selu')(hidden_B)
concat = keras.layers.Concatenate()((hidden_A, hidden_B))
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

X1.shape:  (None, 4)  X2.shape:  (None, 4)


In [59]:
model.compile(loss='mse', optimizer='nadam')

In [60]:
model.fit((X_train_scaled_A, X_train_scaled_B), y_train, epochs=2,
          validation_data=((X_valid_scaled_A, X_valid_scaled_B), y_valid))

Epoch 1/2
X1.shape:  (None, 4)  X2.shape:  (None, 4)
X1.shape:  (None, 4)  X2.shape:  (None, 4)
363/363 [==============================] - 1s 2ms/step - loss: 2.1142 - val_loss: 1.3630
Epoch 2/2
363/363 [==============================] - 1s 3ms/step - loss: 0.9684 - val_loss: 0.9773


If the layer needs to behave differently during training and during testing (this maybe because of Dropout or BatchNormalization layers), we should add a training argument to the `call()` method. For instance, let's add gaussian noise during training but not during testing.

In [61]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [62]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    AddGaussianNoise(stddev=1.0),
    keras.layers.Dense(30, activation="selu"),
    keras.layers.Dense(1)
])

In [63]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 2.3857 - val_loss: 7.6082
Epoch 2/2
162/162 [==============================] - 0s 1ms/step - loss: 0.7560


0.7559615969657898

Keras has a layer that does the same thing, keras.layers.GaussianNoise

## Customization Examples

In this part, we will have 2 examples from [Géron, A. (2019)](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/). In the first one, we will have dense layers and residual layers which are composed of two dense layers. In the second one, we will go through an important way that we can use when we would like to monitor the internal aspect of our model or we can use it for regularization.

### Customizing the model 1

In [64]:
X_new_scaled = X_test_scaled

In [65]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

We defined our Residual Layer and now we can add it to our main network.

In [66]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

Looks good. Now it is time to try out this neural network.

In [67]:
model = ResidualRegressor(1)
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Epoch 1/5
363/363 [==============================] - 0s 1ms/step - loss: 10.5685
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 3.7485
Epoch 3/5
363/363 [==============================] - 1s 2ms/step - loss: 0.8561
Epoch 4/5
363/363 [==============================] - 1s 1ms/step - loss: 0.5982
Epoch 5/5
162/162 [==============================] - 0s 2ms/step - loss: 1.1461


Let's save it and load then.

In [68]:
model.save("my_custom_model.ckpt")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_custom_model.ckpt\assets


In [69]:
model = keras.models.load_model("my_custom_model.ckpt")

In [70]:
history = model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
363/363 [==============================] - 0s 1ms/step - loss: 0.8188
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 0.6153
Epoch 3/5
363/363 [==============================] - 1s 2ms/step - loss: 0.7590
Epoch 4/5
363/363 [==============================] - 0s 1ms/step - loss: 0.7761
Epoch 5/5
363/363 [==============================] - 1s 2ms/step - loss: 0.4848


Okay so far so good, now another interesting question is that how we can implement the same network by using sequential API. Let's see how it works

In [71]:
block1 = ResidualBlock(2, 30)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal"),
    block1, block1, block1, block1,
    ResidualBlock(2, 30),
    keras.layers.Dense(1)
])

In [72]:
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Epoch 1/5
363/363 [==============================] - 0s 1ms/step - loss: 0.9951
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 0.6544
Epoch 3/5
363/363 [==============================] - 0s 1ms/step - loss: 0.8086
Epoch 4/5
363/363 [==============================] - 0s 953us/step - loss: 0.4186
Epoch 5/5
162/162 [==============================] - 0s 672us/step - loss: 0.7437


### Customizing the Model 2

The network below will have two different error measurements. The first one is reconstruction error between the inputs and auxiliary output on top of the upper hidden layer. The other error (I will try to do that) will be the error calculated after all the hidden layers. 

In [73]:
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        self.reconstruct = keras.layers.Dense(8) # workaround for TF issue #46858
        self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")
        self.hidden_mean=keras.metrics.Mean(name="hidden_error")


    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        hidden_loss=tf.reduce_mean(tf.square(self.out(Z) - inputs))
        self.add_loss(0.015 * recon_loss)
        self.add_loss(0.001*hidden_loss)
        if training:
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(self.reconstruction_mean(recon_loss))
        self.add_metric(self.hidden_mean(hidden_loss))
        return self.out(Z)

In [74]:
model = ReconstructingRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2)
y_pred = model.predict(X_test_scaled)

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 0.8162 - reconstruction_error: 1.0187 - hidden_error: 5.7070
Epoch 2/2
363/363 [==============================] - 0s 934us/step - loss: 0.4229 - reconstruction_error: 0.4107 - hidden_error: 6.0958


I wanted to calculate the error after hidden layers but probably I made a mistake (or maybe I didn't, I'm not sure). Additional error measurements can be used for two purposes: 

1. We can use them to add an slight regularization affect on our network.
2. We can also use them for monitoring the internal situation.

In [75]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_pred,y_test)

0.36321304879237987

## Custom Training Loops

In some cases, one may need to define all the training loops. One example of this rare case is Wide&Deep Network architecture in which we have two different optimization functions. I will go into details of creating a custom training loop in the following notebooks, however, let's make a quick introduction here.

Let's start with a function that randomly samples a batch of instances.

In [76]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

Let's also define a function that will inform us about the training status.

In [77]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics,
          end=end)

In [78]:
import time

mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 - loss: 0.0900 - mean_square: 858.5000


We can define some functions that will make training look a bit fancier.

In [79]:
def progress_bar(iteration, total, size=30):
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

In [80]:
def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)

In [81]:
mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 [==============================] - loss: 0.0900 - mean_square: 858.5000


Now let's define the hyperparameters.

In [82]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanSquaredError()]

Let's build the custom loop.

In [83]:
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

11610/11610 [==============================] - mean: 0.5393 - mean_squared_error: 0.5258
Epoch 2/5
11610/11610 [==============================] - mean: 0.4399 - mean_squared_error: 0.4280
Epoch 3/5
11610/11610 [==============================] - mean: 0.3547 - mean_squared_error: 0.3440
Epoch 4/5
11610/11610 [==============================] - mean: 0.3621 - mean_squared_error: 0.3506
Epoch 5/5
11610/11610 [==============================] - mean: 0.3646 - mean_squared_error: 0.3517


We can improve the model a bit and make it look even fancier.

In [84]:
try:
    from tqdm.notebook import trange
    from collections import OrderedDict
    with trange(1, n_epochs + 1, desc="All epochs") as epochs:
        for epoch in epochs:
            with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epoch, n_epochs)) as steps:
                for step in steps:
                    X_batch, y_batch = random_batch(X_train_scaled, y_train)
                    with tf.GradientTape() as tape:
                        y_pred = model(X_batch)
                        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                        loss = tf.add_n([main_loss] + model.losses)
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    for variable in model.variables:
                        if variable.constraint is not None:
                            variable.assign(variable.constraint(variable))                    
                    status = OrderedDict()
                    mean_loss(loss)
                    status["loss"] = mean_loss.result().numpy()
                    for metric in metrics:
                        metric(y_batch, y_pred)
                        status[metric.name] = metric.result().numpy()
                    steps.set_postfix(status)
            for metric in [mean_loss] + metrics:
                metric.reset_states()
except ImportError as ex:
    print("To run this cell, please install tqdm, ipywidgets and restart Jupyter")

All epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 2/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 3/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 5/5:   0%|          | 0/362 [00:00<?, ?it/s]

What's happening in these two functions ?

1.	We have two nested loops. The first one is for epochs, whereas the other one is for the batches in each epoch.
2.	We start by sampling a random batch and then inside `tf.GradientTape()` block we make a prediction and compute the loss.
3.	We use `tf.reduce_mean()` for calculating the mean loss over the batch. Afterwards, we sum the losses (main loss + regularization loss) with `tf.add_n()`
4.	Then we the tape computes the gradients(with regard to each trainable variable) and we apply to the optimizer.
5.	Lastly, we update the mean loss and the metrics over the current epoch.